<center>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width="1000" alt="cognitiveclass.ai logo"  />
</center>

# Instructional Lab:  Create a Db2 asset in your IBM Watson Studio project and load data into the database

Estimated time needed: 45 minutes

# Introduction

Using this R notebook you will complete the following tasks:

1.  Create a Db2 'connection asset' in your IBM Watson Studio project
2.  Create a token for your project
3.  Insert the project token into your Jupyter notebook
4.  Use the 'project-Lib' library to fetch your Db2 connection credentials
5.  Establish your Db2 connection
6.  Load the datasets into Db2

You will not be graded in this instructional lab.\
However, you will be asked to query the Db2 tables you create here in the next lab, which contains graded tasks.


Let's begin by going through steps 1-5 for enabling a connection to Db2 from a Jupyter Notebook running an R kernel within your IBM Watson Studio Project.

## Step 1. Create a Db2 'connection asset' in your IBM Watson Studio project

Click '**Add to project +**' and select the '**Connection**' asset type:

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/module%203/images/add_connection_asset.png" width="1200" alt=""  />

Select '**Db2**' as the type of data source:

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/module%203/images/db2_asset.png" width="1200" alt=""  />

This brings up a new window where you can now enter a name for your connection, the database details, and your credentials. Use the same information that you previously used to connect to Db2 in the labs from **Course 3: SQL for Data Science Using R**.\
Now click '**Create**'. Your new connection should be listed under '**Data assets**' on the Assets page for your IBM Watson Studio project.

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/module%203/images/newDb2connection.png" width="1200" alt=""  />

## Step 2. Create a token for your project

On the settings page for your project, click on '**New Token +**'. Then enter a Token name and set the access role to '**Editor**', then click '**Create**' <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/module%203/images/NewToken.png" width="1200" alt=""  />

## Step 3. Insert the project token into your Jupyter notebook

Click on the icon above your notebook with the three dots, then click 'Insert project token'. This will add a cell to the top of your notebook that you will need to run to enable access to your IBM Watson Studio project resources, and in particular, your Db2 connection. The resulting automatically generated cell should look like this:

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/module%203/images/InsertProjectToken.png" width="1200" alt=""  />


## Step 4. Use the 'project-Lib' library to fetch connection credentials

The projectLib library should now be loaded as a result of running the above auto-generated project token cell. Also, the 'project' variable contains everything you need to connect to Db2 from R. Now run the following code to get your connection credentials:


In [1]:
# Fetch connection
# conn.cred <- project$get_connection(name="Db2Connection") # use your connection name in place of "Db2Connection"
# conn.cred

## Step 5. Establish your Db2 connection

Load the 'RODBC' library and use the 'odbcConnect( )' function to establish the connection. Once you run the following cell (as is), you will be ready to start running SQL queries using the RODBC library as you did in Course 3.


In [2]:
# library(RODBC)
# props <- paste("DASHDB;DATABASE=BLUDB;HOSTNAME=", conn.cred$host, ";PORT=50000;PROTOCOL=TCPIP;", sep="")
# conn <- odbcConnect(props, uid = conn.cred$username, pwd = conn.cred$password)

In [3]:
# Dump connection info
##############################################################
# sql.info <- sqlTypeInfo(conn)
# conn.info <- odbcGetInfo(conn)
# conn.info["DBMS_Name"]
# conn.info["DBMS_Ver"]
# conn.info["Driver_ODBC_Ver"]

In [4]:
import pandas as pd
import sqlite3
import sqlite3 as sq3

In [5]:
%%capture
%load_ext sql
%sql sqlite://bike.db

In [6]:
con = sq3.connect("bike.db")

In [7]:
con

In [8]:
cursor = con.cursor()

## Step 6. Load the Datasets into Db2

Now let's prepare to load the datasets into Db2 tables.

### Dataset URLs

Use the following urls to load your datasets into R dataframes. The datasets are already clean, but you will still need to pay careful attention to data types, especially dates, which you may need to coerce. Also, ensure any categorical variables get typed as factors.

```RR
seoul_bike_sharing <- "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/datasets/seoul_bike_sharing.csv"  

cities_weather_forecast <- "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/datasets/cities_weather_forecast.csv"  

bike_sharing_systems <- "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/datasets/bike_sharing_systems.csv"  

world_cities <- "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/datasets/world_cities.csv"
```

Name the tables as follows:

### Table Names

1.  SEOUL_BIKE_SHARING
2.  CITIES_WEATHER_FORECAST
3.  BIKE_SHARING_SYSTEMS
4.  WORLD_CITIES


### Check whether the tables already exist, and drop them if so.


### Dataset reading and table loading considerations

Now you will read each dataset into an R dataframe using the urls provided above, and finally, load your tables.

Instead of first creating a table and specifying the column names and types as we did in the previous SQL course with SN Labs, the IBM Watson Studio environment allows us the convenience to proceed directly to creating and populating the table simultaneously, using the `sqlSave()` function from the `RODBC` package in R.

We will still set `colClasses=c(Date="character")` when reading data with Date columns into a dataframe, but a couple of things will differ from how we accomplished the data loading in the SN Labs environment:

1.  We will need to cast the Date column of the dataframe as type `Date`
2.  When we load the dataframe into a table using `sqlSave()`, we need to specify `varTypes=c(Date="Date")`

This explicitly ensures that Db2 interprets the `Date` column as a date type, so we can later use built-in Db2 date functions.


## Read the datasets into dataframes


In [9]:
seoul_bike_sharing = pd.read_csv("seoul_bike_sharing_clean.csv")
cities_weather_forecast = pd.read_csv("cities_weather_forecast_clean.csv")
bike_sharing_systems = pd.read_csv("bike_sharing_systems_clean.csv")
world_cities = pd.read_csv("world_cities_clean.csv")

In [ ]:
seoul_bike_sharing.to_sql("SEOUL_BIKE_SHARING", con, index=False)  #Load to database

In [ ]:
cities_weather_forecast.to_sql("CITIES_WEATHER_FORECAST", con, index=False)  #Load to database

In [ ]:
bike_sharing_systems.to_sql("BIKE_SHARING_SYSTEMS", con, index=False)  #Load to database

In [ ]:
world_cities.to_sql("WORLD_CITIES", con, index=False)  #Load to database

In [10]:
con.execute("Select * FROM sqlite_master").fetchall()

[('table',
  'SEOUL_BIKE_SHARING',
  'SEOUL_BIKE_SHARING',
  2,
  'CREATE TABLE "SEOUL_BIKE_SHARING" (\n"DATE" TEXT,\n  "RENTED_BIKE_COUNT" INTEGER,\n  "HOUR" INTEGER,\n  "TEMPERATURE" REAL,\n  "HUMIDITY" INTEGER,\n  "WIND_SPEED" REAL,\n  "VISIBILITY" INTEGER,\n  "DEW_POINT_TEMPERATURE" REAL,\n  "SOLAR_RADIATION" REAL,\n  "RAINFALL" REAL,\n  "SNOWFALL" REAL,\n  "SEASONS" TEXT,\n  "HOLIDAY" TEXT,\n  "FUNCTIONING_DAY" TEXT\n)'),
 ('table',
  'CITIES_WEATHER_FORECAST',
  'CITIES_WEATHER_FORECAST',
  173,
  'CREATE TABLE "CITIES_WEATHER_FORECAST" (\n"CITY" TEXT,\n  "WEATHER" TEXT,\n  "VISIBILITY" INTEGER,\n  "TEMP" REAL,\n  "TEMP_MIN" REAL,\n  "TEMP_MAX" REAL,\n  "PRESSURE" INTEGER,\n  "HUMIDITY" INTEGER,\n  "WIND_SPEED" REAL,\n  "WIND_DEG" INTEGER,\n  "SEASON" TEXT,\n  "FORECAST_DATETIME" TEXT\n)'),
 ('table',
  'BIKE_SHARING_SYSTEMS',
  'BIKE_SHARING_SYSTEMS',
  178,
  'CREATE TABLE "BIKE_SHARING_SYSTEMS" (\n"COUNTRY" TEXT,\n  "CITY" TEXT,\n  "SYSTEM" TEXT,\n  "BICYCLES" REAL\n)'),
 ('ta

In [11]:
pd.read_sql_query("Select * FROM sqlite_master", con) #Check all tables inside database

,type,name,tbl_name,rootpage,sql
0,table,SEOUL_BIKE_SHARING,SEOUL_BIKE_SHARING,2,"CREATE TABLE ""SEOUL_BIKE_SHARING"" (\n""DATE"" TE..."
1,table,CITIES_WEATHER_FORECAST,CITIES_WEATHER_FORECAST,173,"CREATE TABLE ""CITIES_WEATHER_FORECAST"" (\n""CIT..."
2,table,BIKE_SHARING_SYSTEMS,BIKE_SHARING_SYSTEMS,178,"CREATE TABLE ""BIKE_SHARING_SYSTEMS"" (\n""COUNTR..."
3,table,WORLD_CITIES,WORLD_CITIES,184,"CREATE TABLE ""WORLD_CITIES"" (\n""CITY"" TEXT,\n ..."


### Cast the Date column from `seoul_bike_sharing` to `Date`

Using the format "%d/%m/%Y", and check that the resulting datatypes are as expected.


In [12]:
# seoul_bike_sharing$DATE <- as.Date(seoul_bike_sharing$DATE, format = "%d/%m/%Y")
# str(seoul_bike_sharing)

### Load the dataframes into the Db2 tables


In [13]:
# sqlSave(conn, seoul_bike_sharing, "SEOUL_BIKE_SHARING", append=TRUE, fast=FALSE, rownames=FALSE, colnames=FALSE, verbose=FALSE,varTypes=c(DATE="Date"))
# sqlSave(conn, cities_weather_forecast, "CITIES_WEATHER_FORECAST", append=TRUE, fast=FALSE, rownames=FALSE, colnames=FALSE, verbose=FALSE)
# sqlSave(conn, bike_sharing_systems, "BIKE_SHARING_SYSTEMS", append=TRUE, fast=FALSE, rownames=FALSE, colnames=FALSE, verbose=FALSE)
# sqlSave(conn, world_cities, "WORLD_CITIES", append=TRUE, fast=FALSE, rownames=FALSE, colnames=FALSE, verbose=FALSE)

### Check that the column `TYPE_NAME`s are appropriate in the resulting tables


In [14]:
pd.read_sql_query("SELECT * FROM SEOUL_BIKE_SHARING", con)

,DATE,RENTED_BIKE_COUNT,HOUR,TEMPERATURE,HUMIDITY,WIND_SPEED,VISIBILITY,DEW_POINT_TEMPERATURE,SOLAR_RADIATION,RAINFALL,SNOWFALL,SEASONS,HOLIDAY,FUNCTIONING_DAY
0,01/12/2017,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
1,01/12/2017,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
2,01/12/2017,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes
3,01/12/2017,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
4,01/12/2017,78,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,Winter,No Holiday,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8460,30/11/2018,1003,19,4.2,34,2.6,1894,-10.3,0.0,0.0,0.0,Autumn,No Holiday,Yes
8461,30/11/2018,764,20,3.4,37,2.3,2000,-9.9,0.0,0.0,0.0,Autumn,No Holiday,Yes
8462,30/11/2018,694,21,2.6,39,0.3,1968,-9.9,0.0,0.0,0.0,Autumn,No Holiday,Yes
8463,30/11/2018,712,22,2.1,41,1.0,1859,-9.8,0.0,0.0,0.0,Autumn,No Holiday,Yes


In [15]:
pd.read_sql_query("SELECT * FROM CITIES_WEATHER_FORECAST", con)

,CITY,WEATHER,VISIBILITY,TEMP,TEMP_MIN,TEMP_MAX,PRESSURE,HUMIDITY,WIND_SPEED,WIND_DEG,SEASON,FORECAST_DATETIME
0,Seoul,Clear,10000,12.32,10.91,12.32,1015,50,2.18,248,Spring,2021-04-16 12:00:00
1,Seoul,Clear,10000,11.48,9.81,11.48,1016,48,1.25,142,Spring,2021-04-16 15:00:00
2,Seoul,Clouds,10000,9.99,8.82,9.99,1015,46,0.94,130,Spring,2021-04-16 18:00:00
3,Seoul,Clouds,10000,7.87,7.87,7.87,1014,46,0.83,31,Spring,2021-04-16 21:00:00
4,Seoul,Clouds,10000,10.09,10.09,10.09,1014,37,1.96,309,Spring,2021-04-17 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
155,Suzhou,Clouds,10000,11.53,11.53,11.53,1018,76,5.75,121,Spring,2021-04-20 21:00:00
156,Suzhou,Clouds,10000,16.21,16.21,16.21,1019,63,7.15,127,Spring,2021-04-21 00:00:00
157,Suzhou,Clear,10000,21.09,21.09,21.09,1019,42,8.12,133,Spring,2021-04-21 03:00:00
158,Suzhou,Clouds,10000,22.23,22.23,22.23,1016,43,8.18,134,Spring,2021-04-21 06:00:00


In [16]:
pd.read_sql_query("SELECT * FROM BIKE_SHARING_SYSTEMS", con)

,COUNTRY,CITY,SYSTEM,BICYCLES
0,Albania,Tirana,None,200.0
1,Argentina,Mendoza,None,40.0
2,Argentina,"San Lorenzo, Santa Fe",Biciudad,80.0
3,Argentina,Buenos Aires,Serttel Brasil,4000.0
4,Argentina,Rosario,None,480.0
...,...,...,...,...
475,United States,"Santa Monica, California",3 Gen. CycleHop and Social Bicycles,500.0
476,United States,"Savannah, Georgia",3 Gen. B-Cycle,16.0
477,United States,"Seattle, Washington",8D,500.0
478,United States,"Spartanburg, South Carolina",3 Gen. B-Cycle,40.0


In [17]:
pd.read_sql_query("SELECT * FROM WORLD_CITIES", con)

,CITY,CITY_ASCII,LAT,LNG,COUNTRY,ISO2,ISO3,ADMIN_NAME,CAPITAL,POPULATION,ID
0,Tokyo,Tokyo,35.6897,139.6922,Japan,JP,JPN,Tōkyō,primary,37977000.0,1.392686e+09
1,Jakarta,Jakarta,-6.2146,106.8451,Indonesia,ID,IDN,Jakarta,primary,34540000.0,1.360771e+09
2,Delhi,Delhi,28.6600,77.2300,India,IN,IND,Delhi,admin,29617000.0,1.356873e+09
3,Mumbai,Mumbai,18.9667,72.8333,India,IN,IND,Mahārāshtra,admin,23355000.0,1.356227e+09
4,Manila,Manila,14.5958,120.9772,Philippines,PH,PHL,Manila,primary,23088000.0,1.608618e+09
...,...,...,...,...,...,...,...,...,...,...,...
26564,Nord,Nord,81.7166,-17.8000,Greenland,GL,GRL,Sermersooq,None,10.0,1.304218e+09
26565,Timmiarmiut,Timmiarmiut,62.5333,-42.2167,Greenland,GL,GRL,Kujalleq,None,10.0,1.304206e+09
26566,Cheremoshna,Cheremoshna,51.3894,30.0989,Ukraine,UA,UKR,Kyyivs’ka Oblast’,None,0.0,1.804043e+09
26567,Ambarchik,Ambarchik,69.6510,162.3336,Russia,RU,RUS,Sakha (Yakutiya),None,0.0,1.643739e+09


Congratulations! Having successfully loaded your datasets into Db2, you are ready to move on to demonstrating your SQL querying skills. (Please close the connection)


In [18]:
con.close()

## Author(s)

<h4> Jeff Grossman </h4>

## Contributor(s)

<h4> Rav Ahuja, Yan Luo </h4>

## Change log

| Date       | Version | Changed by    | Change Description                                                                                          |
| ---------- | ------- | ------------- | ----------------------------------------------------------------------------------------------------------- |
| 2021-04-22 | 0.2     | Jeff Grossman | Updated based on peer review                                                                                |
| 2021-04-05 | 0.1     | Jeff Grossman | Started content creation                                                                                    |

## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
